# Tensorflow Graphs are just protobufs.

(First post [here](https://medium.com/@ouwenhuang/how-to-learn-tensorflow-the-hard-way-8b34b1e8e173)). In a Tensorflow computational graph, nodes are [`tf.Operation`](https://www.tensorflow.org/versions/master/api_docs/python/tf/Operation) objects and edges pass [`tf.Tensor`](https://www.tensorflow.org/versions/master/api_docs/python/tf/Tensor) objects. A `tf.Operation` then takes `tf.Tensor` objects as inputs and outputs.

<img src="../figures/graph_update.png" width="700"/>
*Figure 1: The figure above is an example of a computational graph. Each node **{A, B, C, D, E, F}** performs some computational operation. **{A}** will square inputs, **{B}** will cube inputs, **{C}** adds one, etc. Each individual node is dependent only on its inputs. The program is created by the way nodes are wired together.
*

Let’s create the computational graph (figure 1) using the mathematical `tf.Operation` objects provided [here](https://www.tensorflow.org/api_docs/python/tf#functions). **Important note: we are only building the graph, not evaluating.** This is equivalent to typing numbers into the calculator without pressing enter.

In [1]:
import tensorflow as tf
import numpy as np
from src import cloud_visualizer

# Let's explicitly create an empty graph: `g1`.
#
# Note, tensorflow has a default graph that can be used but we 
# explicitly create `g1` for clarity.
g1 = tf.Graph()

# `my_input_value` is a tensor-like object. 
# We provide a simple scalar and a numpy array as examples.

# my_input_value = np.random.multivariate_normal(
#     mean=(1,1),
#     cov=[[1,0], [0,1]],
#     size=10
# )

# simple scalar value
my_input_value = 2

# We want our operations to be placed on `g1` and not the default graph.
with g1.as_default():
    
    # Tensorflow operations usually take in a Tensor-like type, a data type, and a name.
    my_input = tf.constant(my_input_value, dtype=tf.float32, name="input")
    
    # These will be implicitly dtype `tf.float16`
    a = tf.square(my_input, name="A")
    b = tf.cos(a, name="B")
    c = tf.sin(a, name="C")   
    d = tf.add(b, c, name="D")
    e = tf.floor(b, name="E")
    f = tf.sqrt(d, name="F")

# We can write the graph as protobuf text file
tf.train.write_graph(graph_or_graph_def=g1, 
                     logdir='/tmp/storage/', 
                     name='graph_protobuf.pbtxt')

# We can write out our graph to tensorboard for visualization
tf.summary.FileWriter("/tmp/logs", g1).close()
cloud_visualizer.show_graph(g1)

# Generate a tensorboard visualization by running the following command
# `tensorboard --logdir=/tmp/logs`

*Every node (including the input constant) is a `tf.Operation`, and every line passes a `tf.Tensor`.*

Each of the operations above can take in a [tensor-like object](https://www.tensorflow.org/programmers_guide/graphs#tensor-like_objects) (scalar, list, numpy array). `tf.Tensor` should feel very similar to python [numpy](https://docs.scipy.org/doc/numpy-dev/user/quickstart.html).

 - They have similar data types which must be conformed to. ([`tf.DType`](https://www.tensorflow.org/api_docs/python/tf/DType): e.g. `tf.float16`, `tf.string`, `tf.variant`)
 - They allow for n-dimensional shapes: [0-dimensional scalar](https://www.tensorflow.org/versions/master/programmers_guide/tensors#rank_0), [1-dimensional vector](https://www.tensorflow.org/versions/master/programmers_guide/tensors#rank_1), [2-dimensional matrix](https://www.tensorflow.org/versions/master/programmers_guide/tensors#higher_ranks), etc.
 - They have a similar [matrix slice notation](https://www.tensorflow.org/api_docs/python/tf/Tensor#__getitem__).
 
However, `tf.Tensor` objects are only symbolic handles, and do not actually provide a concrete value until evaluated. It is a redeemable prize ticket, not the prize itself.

We will use [tensorboard](https://www.tensorflow.org/guide/summaries_and_tensorboard#launching_tensorboard), a graph visualization tool to view our built graph. (run the above cell)

We have now defined a graph which can be saved in a [protocol buffer text](https://developers.google.com/protocol-buffers/) format. Protocol buffers are just like any other structure notation such as JSON, XML, HTML. Protobuf is a preferred choice because they are very compact, and are strongly typed. Tensorflow graphs use the protobuf GraphDef to define a graph. This `GraphDef` protobuf can be exported and imported.

We can view graph `g1` by running `g1.as_graph_def()` this will output the text form of the graph, which we view below.

<img src="../figures/code.gif" width="700"/>
*This protobuf contains the numpy matrix input graph, the full ascii video can be viewed [here](https://asciinema.org/a/177052). The simple float32 gist in this article can be found [here](https://gist.github.com/Ouwen/e656b9114ba146a49b2a0c2d870e5049).*

This protobuf file contains everything needed to reconstruct a tensorflow graph. You can load in the `graph_protobuf.pbtxt` to retrieve the program. Changing the internals of this file is analogous to programming a new graph program.


In [2]:
import tensorflow as tf
from google.protobuf import text_format

# Let's read our pbtxt file into a Graph protobuf
f = open("/tmp/storage/graph_protobuf.pbtxt", "r")
graph_protobuf = text_format.Parse(f.read(), tf.GraphDef())

# Import the graph protobuf into our new graph.
graph_clone = tf.Graph()
with graph_clone.as_default():
    tf.import_graph_def(graph_def=graph_protobuf, name="")

# Display the graph inline.
graph_clone.as_graph_def()

node {
  name: "input"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
        }
        float_val: 2.0
      }
    }
  }
}
node {
  name: "A"
  op: "Square"
  input: "input"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
}
node {
  name: "B"
  op: "Cos"
  input: "A"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
}
node {
  name: "C"
  op: "Sin"
  input: "A"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
}
node {
  name: "D"
  op: "Add"
  input: "B"
  input: "C"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
}
node {
  name: "E"
  op: "Floor"
  input: "B"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
}
node {
  name: "F"
  op: "Sqrt"
  input: "D"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
}
versions {
  producer: 24
}

So far we have only been admiring our graph program, but we have not actually run it yet. We will do so with the tensorflow Session API [here](https://medium.com/@ouwenhuang/tensorflow-sessions-statically-run-tensorflow-graphs-1075ef346783).